<a href="https://colab.research.google.com/github/sunny-5555/Computer-Vision-Assignments/blob/update-assignment/a4-cnn/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Vision - Assignment 4: Image Classification with Convolutional Neural Networks

---

Prof. Dr. Markus Enzweiler, Esslingen University of Applied Sciences

markus.enzweiler@hs-esslingen.de

---

This is the fourth assignment for the "Machine Vision" lecture. 
It covers:
* training a deep CNN from scratch on [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)
* evaluating the effects of different optimizers and regularization
* finetuning an existing CNN on [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)

**Make sure that "GPU" is selected in Runtime -> Change runtime type**

To successfully complete this assignment, it is assumed that you already have some experience in Python and numpy. You can either use [Google Colab](https://colab.research.google.com/) for free with a private (dedicated) Google account (recommended) or a local Jupyter installation.

---


## Preparations


### Import important libraries (you should probably start with these lines all the time ...)

In [ ]:
# OpenCV
import cv2   

# NumPy                    
import numpy as np   

# Python stuff
import glob, urllib, os, requests

# Matplotlib    
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# make sure we show all plots directly below each cell
%matplotlib inline 

# Some Colab specific packages
if 'google.colab' in str(get_ipython()):
  # image display
  from google.colab.patches import cv2_imshow 

# Tensorflow and Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Input, Lambda
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image

# Check the GPU that we got from Colab
!nvidia-smi 
device_name = tf.test.gpu_device_name()
print("Device used for TensorFlow : {}".format(device_name))


### Some helper functions that we will need

In [ ]:
def my_imshow(image, windowTitle="Image"):
  '''
  Displays an image and differentiates between Google Colab and a local Python installation. 

  Args: 
    image: The image to be displayed

  Returns:
    - 
  '''

  if 'google.colab' in str(get_ipython()):
    cv2_imshow(image)
  else:
    cv2.imshow(windowTitle, image)

## Exercise 1 - Define and train a (small) CNN on CIFAR-10 (10 points) 

In this exercise you will be defining and training a small deep CNN on CIFAR-10. We will build on the previous assignment, where a fully connected multilayer perceptron has been trained on CIFAR-10. Its performance on the validation dataset was approx. 53% which is not a stellar performance. The CNN will significantly improve performance over the multilayer perceptron.

Additionally, the effect of different optimizers and regularization techniques will be evaluated step-by-step. 

We will be using TensorFlow and [Keras](https://keras.io/), a high-level API built on top of TensorFlow that provides an easier API to the training of neural networks in comparison to plain TensorFlow.

### Getting familiar with the CIFAR-10 dataset (**PROVIDED**)

In [ ]:
# CIFAR-10 is available as standard dataset in Keras. Nice :)


# load the data
(trainSamples, _trainLabels), (testSamples, _testLabels) = cifar10.load_data()

# scale the image data to float 0-1 (always recommended with neural networks)
trainSamples = trainSamples.astype('float32') / 255.0 
testSamples  = testSamples.astype('float32') / 255.0 

# convert a class vector (integers) to binary class matrix.
trainLabels  = to_categorical(_trainLabels)
testLabels   = to_categorical(_testLabels)

# text representation of class labels
classNames = ['airplane', 'automobile', 'bird', \
               'cat', 'deer', 'dog', \
               'frog', 'horse', 'ship', 'truck']

# Visualize 25 random images
plt.figure(figsize=(10,10))
indices = np.arange(len(trainSamples))
np.random.shuffle(indices)
count=0
for i in indices[0:25]:
    plt.subplot(5,5,count+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(trainSamples[i], cmap=plt.cm.binary)
    plt.xlabel("label: {}".format(classNames[np.argmax(trainLabels[i])]))
    count = count+1
plt.show()

### CNN Model Definition (**add your code here**)

We want to design a standard "feed-forward" CNN. In Keras-terms, this is referred to as a [sequential model](https://www.tensorflow.org/guide/keras/sequential_model). The basic TensorFlow tutorial on [Convolutional Neural Networks](https://www.tensorflow.org/tutorials/images/cnn) is a good resource to learn how CNNs are defined and trained. 

We will need the following layers (input to output):
* 1 [Input](https://www.tensorflow.org/api_docs/python/tf/keras/Input) layer with ```shape = (32,32,3)``` that inputs our 32x32x3 image into the CNN


* 2 [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) convolutional layers with **32** filters of size 3x3, ```relu``` activation functions, ```he_uniform``` kernel initializers and ```same``` padding
* 1 [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layer with 2x2 pooling

* 2 [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) convolutional layers with **64** filters of size 3x3, ```relu``` activation functions, ```he_uniform``` kernel initializers and ```same``` padding
* 1 [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layer with 2x2 pooling

* 2 [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) convolutional layers with **128** filters of size 3x3, ```relu``` activation functions, ```he_uniform``` kernel initializers and ```same``` padding
* 1 [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layer with 2x2 pooling


* 1 [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten) layer to flatten the input for the upcoming fully connected layer 

* 1 [Dense](https://keras.io/api/layers/core_layers/dense/) fully connected layer with 128 neurons and ```relu``` activation and  ```he_uniform``` kernel initializers

* 1 [Dense](https://keras.io/api/layers/core_layers/dense/) fully connected output layer with 10 neurons (1 per class) and ```softmax``` activation. 


Your ```model.summary()``` should look as follows (layer indices might differ). Notice how the representation shrinks with each pooling layer.

```_________________________________________________________________
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 128)         147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
=================================================================
Total params: 550,570
Trainable params: 550,570
Non-trainable params: 0

```

In [ ]:
model = Sequential()

# Define the layers of the CNN model

# input layer
model.add(tf.keras.layers.InputLayer(input_shape=(32, 32, 3)))
# 2 conv layers and 1 pooling layer 
# convolutional layers with 32 filters of size 3x3, relu activation functions, he_uniform kernel initializers and same padding
# pooling layer with 2x2 pooling
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPool2D(pool_size=(2, 2)))
# 2 conv layers and 1 pooling layer
# convolutional layers with 64 filters of size 3x3, relu activation functions, he_uniform kernel initializers and same padding
# pooling layer with 2x2 pooling
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPool2D(pool_size=(2, 2)))
# 2 conv layers and 1 pooling layer
# convolutional layers with 128 filters of size 3x3, relu activation functions, he_uniform kernel initializers and same padding
# pooling layer with 2x2 pooling
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPool2D(pool_size=(2, 2)))
# fully connected layer
# layer to flatten the input for the upcoming fully connected layer
# fully connected layer with 128 neurons and relu activation and he_uniform kernel initializers
model.add(Flatten())
model.add(Dense(units=128, activation='relu', kernel_initializer='he_uniform'))
# output layer
# fully connected output layer with 10 neurons (1 per class) and softmax activation
model.add(Dense(units=10, activation='softmax'))


print(model.summary())

### CNN training with different optimizers (**add your code here**)

Compile the model ([model.compile()](https://keras.io/api/models/model_training_apis/)) and use ```categorical_crossentropy``` as loss and ```accuracy``` as metric (see previous assignment). 


Train your CNN using [model.fit()](https://keras.io/api/models/model_training_apis/). Pass ```trainSamples```and ```trainLabels```as training set and ```testSamples```and ```testLabels``` as ```validation_data```. 

Use the following hyper-parameters:
* ```batch_size = 64```
* ```epochs = 30``` 
* ```verbose = 1```

Run the training three times and switch optimizers with each training run by passing different optimizers to [model.compile()](https://keras.io/api/models/model_training_apis/) (all other settings remain the same):
* Stochastic Gradient Descent (plain): ```optimizer=SGD(learning_rate=3e-4)```
* RMSprop (with momentum): ```optimizer=RMSprop(learning_rate=3e-4)```
* Adam: ```optimizer=Adam(learning_rate=3e-4)```

Different optimizers might need different numbers of training epochs (hyper parameter !!). To find a good number of training epochs for each optimizer, we can use [tf.keras.callbacks.EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) to stop training automatically based on a performance metric that is evaluated during training. Browse through [tf.keras.callbacks.EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) and add ```EarlyStopping``` as a callback to [model.fit()](https://keras.io/api/models/model_training_apis/) to stop training when ```val_loss```, the current loss on the test set, did not improve for 3 epochs.   

The overall training should take between 7 to 15 seconds per epoch (**on a GPU**). Training times depend on the GPU that we have been assigned by Colab. Reported accuracies on the test data should be approx. 74% with the best optimizer variant. 

Compare the different results by computing the accuracy on the test set using:
```
# evaluate model
_, acc = model.evaluate(testSamples, testLabels, verbose=1)
print("Accuracy = {}".format(acc))
```

 **Which optimizer gave the best accuracy on the test set?**


In [ ]:
# make deep copies of the original (initialized) model to make sure to always train from scratch
modelSGD     = tf.keras.models.clone_model(model)
modelRMS     = tf.keras.models.clone_model(model)
modelAdam    = tf.keras.models.clone_model(model)


callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Stochastic Gradient Descent (plain)
modelSGD.compile(optimizer=SGD(learning_rate=3e-4),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

history = modelSGD.fit(x=trainSamples, y=trainLabels, batch_size=64, epochs=30, verbose=1, callbacks=[callback], validation_data=(testSamples, testLabels))

_, acc = modelSGD.evaluate(testSamples, testLabels, verbose=1)
print("Accuracy = {}".format(acc))

# RMSprop (with momentum)
modelRMS.compile(optimizer=RMSprop(learning_rate=3e-4),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

history = modelRMS.fit(x=trainSamples, y=trainLabels, batch_size=64, epochs=30, verbose=1, callbacks=[callback], validation_data=(testSamples, testLabels))

_, acc = modelRMS.evaluate(testSamples, testLabels, verbose=1)
print("Accuracy = {}".format(acc))

# Adam
modelAdam.compile(optimizer=Adam(learning_rate=3e-4),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

history = modelAdam.fit(x=trainSamples, y=trainLabels, batch_size=64, epochs=30, verbose=1, callbacks=[callback], validation_data=(testSamples, testLabels))

_, acc = modelAdam.evaluate(testSamples, testLabels, verbose=1)
print("Accuracy = {}".format(acc))

### Adding regularization (**add your code here**)

Now we add some regularization to our CNN in terms of [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers with a ```rate``` of 0.33, i.e. 33% of the neurons will be randomly disabled in each batch.  

Add a [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layer after every MaxPool2D layer in your model and retrain using the optimizer than performed best in the previous evaluation. 

In [ ]:
modelDropOut = Sequential()

# Define the layers of the CNN model with dropout
# input layer
modelDropOut.add(tf.keras.layers.InputLayer(input_shape=(32, 32, 3)))

# 2 conv layers and 1 pooling layer 
# convolutional layers with 32 filters of size 3x3, relu activation functions, he_uniform kernel initializers and same padding
modelDropOut.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
modelDropOut.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# pooling layer with 2x2 pooling
modelDropOut.add(MaxPool2D(pool_size=(2, 2)))
# layers with a rate of 0.33, i.e. 33% of the neurons will be randomly disabled in each batch
modelDropOut.add(Dropout(rate=0.33))

# 2 conv layers and 1 pooling layer
# convolutional layers with 64 filters of size 3x3, relu activation functions, he_uniform kernel initializers and same padding
modelDropOut.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
modelDropOut.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# pooling layer with 2x2 pooling
modelDropOut.add(MaxPool2D(pool_size=(2, 2)))
# layers with a rate of 0.33, i.e. 33% of the neurons will be randomly disabled in each batch
modelDropOut.add(Dropout(rate=0.33))

# 2 conv layers and 1 pooling layer
# convolutional layers with 128 filters of size 3x3, relu activation functions, he_uniform kernel initializers and same padding
modelDropOut.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
modelDropOut.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# pooling layer with 2x2 pooling
modelDropOut.add(MaxPool2D(pool_size=(2, 2)))
# layers with a rate of 0.33, i.e. 33% of the neurons will be randomly disabled in each batch
modelDropOut.add(Dropout(rate=0.33))

# fully connected layer
# layer to flatten the input for the upcoming fully connected layer
# fully connected layer with 128 neurons and relu activation and he_uniform kernel initializers
modelDropOut.add(Flatten())
modelDropOut.add(Dense(units=128, activation='relu', kernel_initializer='he_uniform'))

# output layer
# fully connected output layer with 10 neurons (1 per class) and softmax activation
modelDropOut.add(Dense(units=10, activation='softmax'))

# Model summary
print(modelDropOut.summary())

# train the CNN
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

modelDropOut.compile(optimizer=Adam(learning_rate=3e-4),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

history = modelDropOut.fit(x=trainSamples, y=trainLabels, batch_size=64, epochs=30, verbose=1, callbacks=[callback], validation_data=(testSamples, testLabels))

_, acc = modelDropOut.evaluate(testSamples, testLabels, verbose=1)
print("Accuracy = {}".format(acc))

### Visualize some predictions (**PROVIDED**)

Now we should have a model that can achieve approx. 80% accuracy on the test set. Let's look at some predictions. 

In [ ]:
def visualizePredictions(model, testSamples, testSamplesUnnormalized):

  # select 50 images randomly from the test set and run them through the CNN
  plt.figure(figsize=(20,10))

  # 50 random images
  indices = np.arange(len(testSamples))
  np.random.shuffle(indices)
  count=0
  for i in indices[0:50]:
      plt.subplot(5,10,count+1)
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)
      plt.imshow(testSamplesUnnormalized[i], cmap=plt.cm.binary)

      # predict MLP (need to reshape 32 x 32 x 3 pixels -> 1 x 3072 pixels)
      prediction = model.predict(np.expand_dims(testSamples[i], axis=0))
    
      # visualize true and predicted labels
      groundTruthLabel = classNames[np.argmax(testLabels[i])]
      predictedLabel   = classNames[np.argmax(prediction)]
      plt.xlabel("T: {} / P: {}".format(groundTruthLabel, predictedLabel))
      count = count+1
  plt.show()

In [ ]:
# visualize predictions of our current model
visualizePredictions(modelDropOut, testSamples, testSamples)

### Visualize some more predictions (**PROVIDED**)

That does look quite good on CIFAR-10's test set. Let's see how the model performs on some random images grabbed from Google. 

In [ ]:
import io
import requests
import zipfile

# path to test images (you will have to modify that)
testDir = "/content/cnn/"

url = 'https://raw.githubusercontent.com/sunny-5555/Computer-Vision-Assignments/main/a4-cnn/data.zip'

response = requests.get(url, allow_redirects = True)
stream = io.BytesIO(response.content)

print("unzipping {}".format(url))

with zipfile.ZipFile(stream, 'r') as zip_ref:
    zip_ref.extractall(testDir)


# some helper functions to load images from a given path and run a prediction

def myPredict(model, image):

  # resize image to model dimensions
  resizedImage = cv2.resize(image, (32,32))

  # run prediction
  x=np.expand_dims(resizedImage, axis=0)
  x=x.astype('float32') / 255
  predictedClasses = model.predict(x)
  return predictedClasses


def getArgMaxLabel(classNames, predictedClasses):
  return classNames[np.argmax(predictedClasses)]


def visualizePredictionsExtImages(model, predictionFunction):
  # get all images in testDir
  files = [testDir+f for f in os.listdir(testDir) if f.startswith("test") and f.endswith(".jpg")]

  for f in files: 
    plt.figure()  
    # load image

    testImage = cv2.imread(f)

    # display image
    plt.imshow(cv2.cvtColor(cv2.resize(testImage, (400,400)), cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.draw()
    
    # evaluate our CNN
    predictedClasses = predictionFunction(model, testImage)
    predictedLabel   = getArgMaxLabel(classNames, predictedClasses)
    plt.title("{} with a probability of {:.3f}%".format(predictedLabel, 100*np.max(predictedClasses)))
    plt.draw()


In [ ]:
# visualize predictions of our current model
visualizePredictionsExtImages(modelDropOut)#, myPredict)

## Exercise 2 - Fine-tuning / transfer learning of a larger CNN on CIFAR-10 (10 points) 

In this exercise you will be [fine-tuning](https://www.tensorflow.org/tutorials/images/transfer_learning) a large CNN that has been trained on ImageNet to the CIFAR-10 dataset. As we have seen in the lecture, this involves replacing the output layer by (a set of) output layers that correspond to our problem. Here: 10 output neurons for the 10 classes of CIFAR-10. 

As a base CNN we will use [InceptionResNet-V2](https://keras.io/api/applications/inceptionresnetv2/). 

### CNN model definition (**PROVIDED**)

We will load a pre-trained InceptionResNet-V2 and replace its output layers by a new stack of fully connected layers. Luckily, common models that have been pre-trained on ImageNet are readily available in [keras.applications](https://keras.io/api/applications/). 


In [ ]:
# 1) Load the base InceptionResNet-V2 that has been trained on imagenet

# We can omit the original ImageNet output layer directly while loading the model. 
# Additionally, we need to upscale our 32,32 to the input that InceptionResNet-V2 requires (160,160,3)

# We will link different parts of our model together:
# input -> upscale(input) -> base_model(upscale) -> base_model.output -> input to our new classification "head"

# define our input tensor: (32x32x3) images
inputs = Input(shape=(32, 32, 3))

# upscale layer to automatically upscale our images to the correct InceptionResNet-V2 input size
upscale = Lambda(lambda x: tf.image.resize_with_pad(x,
                                                    160,
                                                    160,
                                                    method=tf.image.ResizeMethod.BILINEAR))(inputs)


# load the base model and set the input to "upscale"
base_model = tf.keras.applications.InceptionResNetV2(include_top=False,
                                                     weights='imagenet',
                                                     input_tensor=upscale,
                                                     input_shape=(160,160,3),
                                                     pooling='max')


# 2) We can choose to enable or disable training of the base model, e.g. only train our new top layers or the whole model
base_model.trainable = False

# 3) Add our custom top layers to classify CIFAR-10
out = base_model.output # we build our new layers at the end of the base model's output
out = Flatten()(out)
out = BatchNormalization()(out)
out = Dense(256, activation='relu')(out)
out = Dropout(0.3)(out)
out = BatchNormalization()(out)
out = Dense(128, activation='relu')(out)
out = Dropout(0.3)(out)
out = BatchNormalization()(out)
out = Dense(64, activation='relu')(out)
out = Dropout(0.3)(out)

# 10 output neurons, 1 for every CIFAR-10 class
out = Dense(10, activation='softmax')(out)


# 4) Define the new model
modelFineTune = tf.keras.models.Model(inputs=inputs, outputs=out)

print(modelFineTune.summary())

### Preprocess the training and test data (**PROVIDED**)

Every pre-trained network in Keras comes with its own preprocessing function that needs to be applied to every training and test image. This preprocessing is the same that has been used to train the initial network on ImageNet. 

**From now on, we will need to use the preprocessed datasets for training and testing.**

---



In [ ]:
# load the data again, since we have applied a different normalization in our first Excercise
(trainSamples, _trainLabels), (testSamples, _testLabels) = cifar10.load_data()

# convert a class vector (integers) to binary class matrix.
trainLabels  = to_categorical(_trainLabels)
testLabels   = to_categorical(_testLabels)

# preprocess training and test samples
trainSamplesPP = tf.keras.applications.inception_resnet_v2.preprocess_input(np.copy(trainSamples))
testSamplesPP  = tf.keras.applications.inception_resnet_v2.preprocess_input(np.copy(testSamples))

# class names
# text representation of class labels
classNames = ['airplane', 'automobile', 'bird', \
               'cat', 'deer', 'dog', \
               'frog', 'horse', 'ship', 'truck']

### CNN Training (top layers only) (**add your code here**)

Train the model using your best optimizer from Exercise 1 and the same loss, metrics and hyper parameters as before, except for the ```learning_rate``` which should be decreased to 1e-4. Limit the maximum number of epochs to 10 by setting ```epochs=10```. Evaluate the model's accuracy on the test set. 

**Depending on the GPU that we are being assigned, one epoch can take up to 5 minutes!**

We can provide [ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) as additional callback to ```model.fit()``` to save our model after every epoch. This comes in very handy when training times increase. And, we can always continue training by loading the saved model and run ```model.fit()``` again. 

**Note, that only our new layers are trained! See the number of trainable vs. non-trainable parameters in the output of ```model.summary()```**. 

In [ ]:
# Create a callback that saves the model's weights after every epoch
checkpointPath = "/content/drive/My Drive/cifar-10-training_inception-resnet-v2/myAwesomeCNN.ckpt"
checkPoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpointPath,
                                                save_weights_only=True,
                                                verbose=1)

# Crate an early stopping callback
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


# Compile the new CNN model
modelFineTune.compile(optimizer=Adam(learning_rate=1e-4),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
# Train the new CNN model (make sure to use both callbacks defined above !!)
history = modelFineTune.fit(x=trainSamplesPP, y=trainLabels, batch_size=64, epochs=10, verbose=1, callbacks=[earlyStopping, checkPoint], validation_data=(testSamplesPP, testLabels))

# Evaluate the accuracy
_, acc = modelFineTune.evaluate(testSamplesPP, testLabels, verbose=1)
print("Accuracy = {}".format(acc))

### Visualize the performance (**PROVIDED**)

In [ ]:
# load the model from the saved checkpoint
modelFineTune.load_weights(tf.train.latest_checkpoint(os.path.dirname(checkpointPath)))

In [ ]:
visualizePredictions(modelFineTune, testSamplesPP, testSamples)

In [ ]:
# for external images, we also need to apply the Inception-ResNet-V2 preprocessing

def myPredictInceptionResNet(model, image):

  # resize image to model dimensions
  resizedImage = cv2.resize(image, (32,32))

  # expand dimensions (i.e. build a tensor from a single image)
  x=np.expand_dims(resizedImage, axis=0)
  
  # apply preprocessing
  x=tf.keras.applications.inception_resnet_v2.preprocess_input(x)
  # and predict 
  predictedClasses = model.predict(x)
  return predictedClasses

In [ ]:
visualizePredictionsExtImages(modelFineTune, myPredictInceptionResNet)

### CNN Training (full CNN) (**add your code here**)

In a final experiment, train the full CNN from top to bottom using the same parameters as before.

**Depending on the GPU that we are being assigned, one epoch can take up to 17 minutes!**

**How does the performance differ?**

In [ ]:
# Define a fully trainable model
base_model.trainable = True

# Replicate the CNN architecture from above and use 'modelFineTuneFull' as a variable for this CNN model. 
out = base_model.output 
out = Flatten()(out)
out = BatchNormalization()(out)
out = Dense(256, activation='relu')(out)
out = Dropout(0.3)(out)
out = BatchNormalization()(out)
out = Dense(128, activation='relu')(out)
out = Dropout(0.3)(out)
out = BatchNormalization()(out)
out = Dense(64, activation='relu')(out)
out = Dropout(0.3)(out)

out = Dense(10, activation='softmax')(out)

modelFineTuneFull = tf.keras.models.Model(inputs=inputs, outputs=out)

# Compile the new CNN model
modelFineTune.compile(optimizer=Adam(learning_rate=1e-4),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
# Train the new CNN model (make sure to use both callbacks defined above !!)
history = modelFineTune.fit(x=trainSamplesPP, y=trainLabels, batch_size=64, epochs=10, verbose=1, callbacks=[earlyStopping, checkPoint], validation_data=(testSamplesPP, testLabels))

# Evaluate the accuracy
_, acc = modelFineTune.evaluate(testSamplesPP, testLabels, verbose=1)
print("Accuracy = {}".format(acc))

### Visualize the performance (**PROVIDED**)

In [ ]:
# load the model from the saved checkpoint
modelFineTuneFull.load_weights(tf.train.latest_checkpoint(os.path.dirname(checkpointPath)))

In [ ]:
visualizePredictions(modelFineTuneFull, testSamplesPP, testSamples)

In [ ]:
visualizePredictionsExtImages(modelFineTuneFull, myPredictInceptionResNet)